# Configuración del diccionario de datos - JSON

## Objetivo
Este Jupyter Notebook tiene como objetivo procesar y reconfigurar un archivo JSON para incorporar análisis de prematuridad.

## Importación de Bibliotecas

In [4]:
import pandas as pd
import json
import re

## Carga de Datos JSON

In [9]:
dictionary = json.load(open("output_json_adjusted.json","rb"))

## Asignar código a las etiquetas

Pongamos código asociado a las etiquetas de las variables categóricas. Esto nos ayudará a procesar la información más adelante. 

In [19]:
def reconfigure_json_safe(data):
    """
    Reconfigura un archivo JSON de forma segura, alterando las listas en [properties][names] para dicionários,
    atribuyendo un índice a cada string, caso [properties][options] sea diferente de cero y exista.
    """
    for key, item in data.items():
        # Verifica si 'properties' y 'options' existen
        if 'properties' in item and 'options' in item['properties'] and item['properties']['options'] != 0:
            # Verifica si 'names' existe y es un diccionario
            if 'names' in item['properties'] and isinstance(item['properties']['names'], dict):
                for language in item['properties']['names']:
                    # Crea un diccionario con índices para cada string
                    item['properties']['names'][language] = {
                        str(index): value for index, value in enumerate(item['properties']['names'][language])
                    }
    return data

reconfigured_data_safe = reconfigure_json_safe(dictionary)
reconfigured_data_safe


{'1018': {'type': 'TEXT',
  'names': {'spanish': 'pas de identificacin',
   'english': 'id country',
   'portuguese': 'pas de identificao',
   'dutch': 'holpas de identificacin',
   'french': 'pays du pice didentit'},
  'properties': {'length': 2, 'upperCase': True}},
 '1019': {'type': 'TEXT',
  'names': {'spanish': 'tipo de identificacin',
   'english': 'id type',
   'portuguese': 'tipo de identificao',
   'dutch': 'holnmero de identidad',
   'french': 'type du pice didentit'},
  'properties': {'length': 3, 'upperCase': True}},
 '1096': {'type': 'LONGTEXT',
  'names': {'spanish': 'notas paciente'},
  'properties': {'length': 15000}},
 '1249': {'type': 'BARCODE',
  'names': {'spanish': 'id externo oculto trinidad y tobago',
   'english': 'external hidden id trinidad and tobago'},
  'properties': {'length': 20}},
 '3001': {'type': 'TEXT',
  'names': {'spanish': '', 'english': 'registration  name of professional'},
  'properties': {'length': 50}},
 '3002': {'type': 'TEXT',
  'names': {'s

## Función para Procesar Nombres
Para evitar futuros problemas de tratamiento de datos, colorearemos todas las etiquetas en minúsculas y eliminaremos los caracteres especiales.

In [20]:
def process_names_corrected(data):
    """
    Procesa recursivamente el JSON para encontrar la propiedad 'names' en cualquier nivel.
    Transforma los datos en minúsculas y elimina caracteres especiales.
    """
    if isinstance(data, dict):
        for key, value in data.items():
            # Se a chave é 'names', processa os valores
            if key == 'names':
                if isinstance(value, dict):
                    for lang, text in value.items():
                        if isinstance(text, dict):
                            # Processa cada valor no dicionário
                            for idx, val in text.items():
                                data[key][lang][idx] = re.sub(r'[^a-zA-Z0-9\s]', '', val.lower())
                        elif isinstance(text, str):
                            # Aplica a mesma lógica para strings diretamente em 'names'
                            data[key][lang] = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
                elif isinstance(value, str):
                    # Aplica a mesma lógica para strings diretamente em 'names'
                    data[key] = re.sub(r'[^a-zA-Z0-9\s]', '', value.lower())
            else:
                # Processa recursivamente
                process_names_corrected(value)
    elif isinstance(data, list):
        for item in data:
            # Processa cada item da lista
            process_names_corrected(item)
    return data


processed_data_corrected = process_names_corrected(reconfigured_data_safe)
processed_data_corrected

{'1018': {'type': 'TEXT',
  'names': {'spanish': 'pas de identificacin',
   'english': 'id country',
   'portuguese': 'pas de identificao',
   'dutch': 'holpas de identificacin',
   'french': 'pays du pice didentit'},
  'properties': {'length': 2, 'upperCase': True}},
 '1019': {'type': 'TEXT',
  'names': {'spanish': 'tipo de identificacin',
   'english': 'id type',
   'portuguese': 'tipo de identificao',
   'dutch': 'holnmero de identidad',
   'french': 'type du pice didentit'},
  'properties': {'length': 3, 'upperCase': True}},
 '1096': {'type': 'LONGTEXT',
  'names': {'spanish': 'notas paciente'},
  'properties': {'length': 15000}},
 '1249': {'type': 'BARCODE',
  'names': {'spanish': 'id externo oculto trinidad y tobago',
   'english': 'external hidden id trinidad and tobago'},
  'properties': {'length': 20}},
 '3001': {'type': 'TEXT',
  'names': {'spanish': '', 'english': 'registration  name of professional'},
  'properties': {'length': 50}},
 '3002': {'type': 'TEXT',
  'names': {'s

## Guardar los Datos Procesados

In [6]:
file_name = 'dictionary_pregnancie.json'
with open(file_name, 'w', encoding='utf-8') as file:
    json.dump(reconfigured_data_safe, file, ensure_ascii=False, indent=4)
